In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#!pip install plotly
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import datetime as dt
from datetime import timedelta
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score,silhouette_samples
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
#import statsmodels.api as sm
#from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
#from fbprophet import Prophet
from sklearn.preprocessing import PolynomialFeatures
#from statsmodels.tsa.stattools import adfuller
#import statsmodels.api as sm
#from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
#from fbprophet import Prophet
from sklearn.preprocessing import PolynomialFeatures
#from statsmodels.tsa.stattools import adfuller
#std=StandardScaler()

In [2]:
#reading data
df = pd.read_csv("all-states-history.csv")
df

,date,state,death,deathConfirmed,deathIncrease,deathProbable,hospitalized,hospitalizedCumulative,hospitalizedCurrently,hospitalizedIncrease,...,totalTestResults,totalTestResultsIncrease,totalTestsAntibody,totalTestsAntigen,totalTestsPeopleAntibody,totalTestsPeopleAntigen,totalTestsPeopleViral,totalTestsPeopleViralIncrease,totalTestsViral,totalTestsViralIncrease
0,2021-03-07,AK,305.0,NaN,0,NaN,1293.0,1293.0,33.0,0,...,1731628.0,0,NaN,NaN,NaN,NaN,NaN,0,1731628.0,0
1,2021-03-07,AL,10148.0,7963.0,-1,2185.0,45976.0,45976.0,494.0,0,...,2323788.0,2347,NaN,NaN,119757.0,NaN,2323788.0,2347,NaN,0
2,2021-03-07,AR,5319.0,4308.0,22,1011.0,14926.0,14926.0,335.0,11,...,2736442.0,3380,NaN,NaN,NaN,481311.0,NaN,0,2736442.0,3380
3,2021-03-07,AS,0.0,NaN,0,NaN,NaN,NaN,NaN,0,...,2140.0,0,NaN,NaN,NaN,NaN,NaN,0,2140.0,0
4,2021-03-07,AZ,16328.0,14403.0,5,1925.0,57907.0,57907.0,963.0,44,...,7908105.0,45110,580569.0,NaN,444089.0,NaN,3842945.0,14856,7908105.0,45110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20775,2020-01-17,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20776,2020-01-16,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20777,2020-01-15,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0
20778,2020-01-14,WA,NaN,NaN,0,NaN,NaN,NaN,NaN,0,...,NaN,0,NaN,NaN,NaN,NaN,NaN,0,NaN,0


In [32]:
daily_deaths = pd.DataFrame({'Deaths': df.groupby('date')['death'].sum()})
daily_deaths
#state_conclude = (df_date.groupby(['state']).agg({'positive':sum,'deathConfirmed':sum,'hospitalized':sum,'death':sum}))
#state_conclude['death_rate'] = state_conclude['death']/state_conclude['positive']
#state_conclude

,Deaths
date,
2020-01-13,0.0
2020-01-14,0.0
2020-01-15,0.0
2020-01-16,0.0
2020-01-17,0.0
...,...
2021-03-03,508665.0
2021-03-04,510408.0
2021-03-05,512629.0


In [26]:
state_conclude.sort_values('death_rate', ascending = False)

,positive,deathConfirmed,hospitalized,death,death_rate
state,,,,,
NJ,105084623.0,4746953.0,9992229.0,5352148.0,0.050932
CT,33618419.0,1172715.0,1908439.0,1584666.0,0.047137
MA,71309732.0,3071210.0,4270426.0,3195657.0,0.044814
NY,216883639.0,0.0,0.0,8854467.0,0.040826
MI,82832274.0,2754990.0,0.0,2892062.0,0.034915
DC,6050063.0,0.0,0.0,207067.0,0.034226
PA,101072365.0,0.0,7233.0,3384139.0,0.033482
LA,61057115.0,1667839.0,0.0,1765795.0,0.028920
MP,24253.0,402.0,968.0,677.0,0.027914
